In [ ]:
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

class NoDecisao:
    def __init__(self, caracteristica=None, limite=None, esquerda=None, 
                 direita=None, valor=None, ganho=None, n_amostras=None):
        self.valor = valor
        self.caracteristica = caracteristica
        self.limite = limite
        self.esquerda = esquerda
        self.direita = direita
        self.ganho = ganho  # Ganho de informação da divisão
        self.n_amostras = n_amostras  # Número de amostras no nó

class ArvoreDecisaoAprimorada:
    def __init__(self, profundidade_maxima=10, min_amostras_divisao=2, 
                 min_ganho_informacao=0.01, tipo='classificacao'):
        self.profundidade_maxima = profundidade_maxima
        self.min_amostras_divisao = min_amostras_divisao
        self.min_ganho_informacao = min_ganho_informacao
        self.tipo = tipo
        self.raiz = None
        self.importancias = None
    
    def ajustar(self, dados, rotulos):
        self.raiz = self._construir_arvore(dados, rotulos)
        self._calcular_importancias()
        return self
    
    # --- MÉTRICAS ---
    
    def _entropia(self, rotulos):
        if len(rotulos) == 0:
            return 0
        _, contagens = np.unique(rotulos, return_counts=True)
        probabilidades = contagens / len(rotulos)
        return -np.sum(probabilidades * np.log2(probabilidades + 1e-8))
    
    def _ganho_informacao(self, rotulos_pai, rotulos_esquerda, rotulos_direita):
        if len(rotulos_pai) == 0:
            return 0
        
        entropia_pai = self._entropia(rotulos_pai)
        peso_esquerda = len(rotulos_esquerda) / len(rotulos_pai)
        peso_direita = len(rotulos_direita) / len(rotulos_pai)
        
        entropia_filhos = (peso_esquerda * self._entropia(rotulos_esquerda) + 
                          peso_direita * self._entropia(rotulos_direita))
        
        return entropia_pai - entropia_filhos
    
    def _impureza_gini(self, rotulos):
        """Alternativa à entropia - mais rápida computacionalmente"""
        if len(rotulos) == 0:
            return 0
        _, contagens = np.unique(rotulos, return_counts=True)
        probabilidades = contagens / len(rotulos)
        return 1 - np.sum(probabilidades ** 2)
    
    # --- DIVISÃO ---
    
    def _melhor_divisao(self, dados, rotulos):
        melhor_ganho = -1
        melhor_caracteristica = None
        melhor_limite = None
        
        n_amostras, n_caracteristicas = dados.shape
        
        for idx_caracteristica in range(n_caracteristicas):
            caracteristica = dados[:, idx_caracteristica]
            
            # Amostragem inteligente de limites
            valores_unicos = np.unique(caracteristica)
            if len(valores_unicos) > 100:  # Para features com muitos valores únicos
                percentis = np.linspace(0, 100, 50)
                limites_candidatos = np.percentile(caracteristica, percentis)
            else:
                limites_candidatos = valores_unicos
            
            for limite in limites_candidatos:
                mascara = caracteristica <= limite
                rotulos_esquerda = rotulos[mascara]
                rotulos_direita = rotulos[~mascara]
                
                if len(rotulos_esquerda) < self.min_amostras_divisao or \
                   len(rotulos_direita) < self.min_amostras_divisao:
                    continue
                
                ganho = self._ganho_informacao(rotulos, rotulos_esquerda, rotulos_direita)
                
                if ganho > melhor_ganho:
                    melhor_ganho = ganho
                    melhor_caracteristica = idx_caracteristica
                    melhor_limite = limite
        
        return melhor_caracteristica, melhor_limite, melhor_ganho
    
    def _valor_no(self, rotulos):
        if len(rotulos) == 0:
            return 0 if self.tipo == 'regressao' else None
        
        if self.tipo == 'classificacao':
            return Counter(rotulos).most_common(1)[0][0]
        else:  # regressao
            return np.mean(rotulos)
    
    def _construir_arvore(self, dados, rotulos, profundidade=0):
        n_amostras = len(rotulos)
        
        # Critérios de parada
        if (profundidade >= self.profundidade_maxima or 
            n_amostras < self.min_amostras_divisao * 2 or
            len(np.unique(rotulos)) == 1):
            
            return NoDecisao(valor=self._valor_no(rotulos), n_amostras=n_amostras)
        
        # Encontrar melhor divisão
        caracteristica, limite, ganho = self._melhor_divisao(dados, rotulos)
        
        # Critério de ganho mínimo
        if caracteristica is None or ganho < self.min_ganho_informacao:
            return NoDecisao(valor=self._valor_no(rotulos), n_amostras=n_amostras)
        
        # Aplicar divisão
        mascara = dados[:, caracteristica] <= limite
        esquerda = self._construir_arvore(dados[mascara], rotulos[mascara], profundidade + 1)
        direita = self._construir_arvore(dados[~mascara], rotulos[~mascara], profundidade + 1)
        
        return NoDecisao(caracteristica=caracteristica, limite=limite,
                        esquerda=esquerda, direita=direita, 
                        ganho=ganho, n_amostras=n_amostras)
    
    # --- FUNCIONALIDADES ---
    
    def _calcular_importancias(self):
        """Calcula importância das features baseada no ganho de informação"""
        if self.raiz is None:
            return
        
        importancias = np.zeros(100)  # Assumindo até 100 features
        
        def _percorrer_arvore(no):
            if no.caracteristica is not None:
                # Importância = ganho * proporção de amostras
                importancia = no.ganho * (no.n_amostras / self.raiz.n_amostras)
                importancias[no.caracteristica] += importancia
                _percorrer_arvore(no.esquerda)
                _percorrer_arvore(no.direita)
        
        _percorrer_arvore(self.raiz)
        # Normalizar importâncias
        soma_importancias = np.sum(importancias)
        if soma_importancias > 0:
            self.importancias = importancias / soma_importancias
    
    def obter_importancias(self, nomes_features=None):
        """Retorna importância das features"""
        if self.importancias is None:
            return {}
        
        if nomes_features is None:
            nomes_features = [f'Feature_{i}' for i in range(len(self.importancias))]
        
        return dict(zip(nomes_features, self.importancias))
    
    def profundidade_arvore(self):
        """Calcula a profundidade real da árvore"""
        def _profundidade(no):
            if no.valor is not None:
                return 0
            return 1 + max(_profundidade(no.esquerda), _profundidade(no.direita))
        
        return _profundidade(self.raiz) if self.raiz else 0
    
    # --- PREVISÃO ---
    
    def prever(self, dados):
        return np.array([self._travessia_arvore(amostra, self.raiz) for amostra in dados])
    
    def prever_proba(self, dados):
        """Para classificação: retorna probabilidades"""
        if self.tipo != 'classificacao':
            raise ValueError("prever_proba só disponível para classificação")
        
        def _probabilidades(amostra, no):
            if no.valor is not None:
                # Retorna distribuição de probabilidades
                return {no.valor: 1.0}
            
            if amostra[no.caracteristica] <= no.limite:
                return _probabilidades(amostra, no.esquerda)
            else:
                return _probabilidades(amostra, no.direita)
        
        return [_probabilidades(amostra, self.raiz) for amostra in dados]
    
    def _travessia_arvore(self, amostra, no):
        if no.valor is not None:
            return no.valor
        if amostra[no.caracteristica] <= no.limite:
            return self._travessia_arvore(amostra, no.esquerda)
        return self._travessia_arvore(amostra, no.direita)

# --- EXEMPLOS PRÁTICOS ---

def exemplo_classificacao():
    """Exemplo com dataset Iris"""
    from sklearn.datasets import load_iris
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score, classification_report
    
    # Carregar dados
    iris = load_iris()
    X, y = iris.data, iris.target
    
    # Dividir treino/teste
    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Criar e treinar árvore
    arvore = ArvoreDecisaoAprimorada(
        profundidade_maxima=5,
        min_amostras_divisao=3,
        min_ganho_informacao=0.01,
        tipo='classificacao'
    )
    
    arvore.ajustar(X_treino, y_treino)
    
    # Previsões
    predicoes = arvore.prever(X_teste)
    acuracia = accuracy_score(y_teste, predicoes)
    
    print(f"✅ Acurácia: {acuracia:.3f}")
    print(f"📊 Profundidade real: {arvore.profundidade_arvore()}")
    print("🎯 Importância das features:")
    print(arvore.obter_importancias(iris.feature_names))
    
    return arvore, acuracia

def exemplo_regressao():
    """Exemplo com dataset de preços de casas"""
    from sklearn.datasets import fetch_california_housing
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_error, r2_score
    
    # Carregar dados
    housing = fetch_california_housing()
    X, y = housing.data, housing.target
    
    # Amostrar para melhor performance
    indices = np.random.choice(len(X), 1000, replace=False)
    X, y = X[indices], y[indices]
    
    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Árvore de regressão
    arvore = ArvoreDecisaoAprimorada(
        profundidade_maxima=8,
        min_amostras_divisao=5,
        tipo='regressao'
    )
    
    arvore.ajustar(X_treino, y_treino)
    predicoes = arvore.prever(X_teste)
    
    mse = mean_squared_error(y_teste, predicoes)
    r2 = r2_score(y_teste, predicoes)
    
    print(f"📈 MSE: {mse:.4f}")
    print(f"📊 R² Score: {r2:.4f}")
    print("🎯 Features mais importantes:")
    print(arvore.obter_importancias(housing.feature_names))
    
    return arvore, r2

if __name__ == "__main__":
    print("🌳 ÁRVORE DE DECISÃO APRIMORADA")
    print("=" * 50)
    
    print("\n1. CLASSIFICAÇÃO (Dataset Iris):")
    arvore_class, acc = exemplo_classificacao()
    
    print("\n2. REGRESSÃO (Dataset California Housing):")
    arvore_reg, r2 = exemplo_regressao()